# 2. Quickstart: Running a single Schwarzschild model

By the end of the notebook, you will have run a Schwarzschild model. This will involve,
1. understanding the configuration file
2. executing commands to create and run a Schwarzschild model
3. plotting some output for this model

## Setup

You should be in the directory ```docs/tutorial_notebooks```. To prepare the input data files, you should first run the CALIFA section of the previous tutorial "Data Preparation for Gauss Hermite kinematics" (``1_data_prep_for_gauss_hermites.ipynb``). If this is the first time you have run this tutorial, this directory should have the following structure, 

```
tutorial_notebooks
    |- ...
    |- NGC6278_input
        |- dynamite_input
            |- aperture.dat
            |- bins.dat
            |- gauss_hermite_kins.ecvs
            |- kinmaps.pdf
            |- mge.ecsv
            |- pafit.pdf
        |- kinmaps.pdf
        |- NGC6278.V1200.rscube_INDOUSv2_SN20_stellar_kin.fits
    |- NGC6278_config.yaml
    |- NGC6278_config_single.yaml
    |- *.ipynb
    |- ...
```

We have provided example input data files and a configution file for the galaxy NGC6278. The input data files are:

- ``mge.ecvs`` - the Multi Gaussian Expansion (MGE) describing the stellar surface density
- ``gauss_hermite_kins.ecsv`` - the kinematics extracted for this galaxy
- ``aperture.dat`` - information about the spatial apertures/bins using for kinematic extraction
- ``bins.dat`` - information about the spatial apertures/bins using for kinematic extraction
- some diagnostic plots

The MGE and kinematics files must be in the form of [Astropy ECSV files](https://docs.astropy.org/en/stable/api/astropy.io.ascii.Ecsv.html). This is a convenient and flexible file format, so we use it wherever possible for storing our input and output data.

We have provided test data for one galaxy. A very basic set of instructions for generating your own input data is as follows,

- fit and MGE to a photometric image e.g. using [mge](http://www-astro.physics.ox.ac.uk/~mxc/software/#mge)
- Voronoi bin your IFU datacube e.g. using [e.g. vorbin](http://www-astro.physics.ox.ac.uk/~mxc/software/#binning)
- extract kinematics from the binned datacube e.g. using [e.g. PPXF](http://www-astro.physics.ox.ac.uk/~mxc/software/#ppxf)
- make the apertures and binfile files. An example script which has created these files is the file ``generate_input_califa.py`` in [this directory](https://github.com/dynamics-of-stellar-systems/triaxschwarz/tree/sabine/schwpy/data_prepare)

In the future, we will provide more examples of test data, and more detailed instructions for preparing your own data.

To get started, let's import DYNAMITE and print the version and installation path,

In [ ]:
import dynamite as dyn

print('DYNAMITE')
print('    version', dyn.__version__)
#print('    installed at ', dyn.__path__) # uncomment for complete DYNAMITE installation path

## Reading the configuration file

The configuration file controls all of the settings that you may wish to vary when running your Schwarschild models, e.g.

- specifying the components of the gravitational potential
- specifying the potential parameters, or parameter ranges, etc
- specify what type of kinematic data you are providing, e.g.
    - discrete vs continuous,
    - Gauss-Hermite vs Histograms
- the location of the input and output files
- the number of models you want to run

This list of options is incomplete - for a more detailed description of the configuration file, see the documentation.

The configuration file for this tutorial is
```
NGC6278_config_single.yaml
```
Open this file in a text editor, alongside this notebook, to see how it is organised. The file is in ``yaml`` format. The basic structure of a yaml files are pairs of keys and values
```
key : value
```
which can be organised into hierarchical levels separated by tabs
```
main_key:
    sub_key1 : value1
    sub_key2 : value2
```
Comments begin with a ``#``. Values can be any type of variable e.g. integers, floats, strings, booleans etc.

To read in the congfiguration file we can use the following command, creating a configuration object which here we call ``c``,

In [ ]:
fname = 'NGC6278_config_single.yaml'
c = dyn.config_reader.Configuration(fname, reset_logging=True)

On making this object, some output is printed telling us whether any previous models have been found. Assuming that you're running this tutorial for the first time, then no models will be found and an empty table is created at ``AllModels.table``. This table holds holds information about all the models which have been run so far.

The configuration object ``c`` is structured in a similar way to the the configuration file itself. For example, the configuration file is split into two sections. The top section defines aspects the physical system we wish to model - e.g. the globular cluster, galaxy or galaxy cluster - while the second section contains all other settings we need for running a model - e.g. settings about the orbit integration and input/output options. The two sections are stored in the ``system`` and ``settings`` attributes of the configuration object, respectively,

In [ ]:
print(type(c.system))
print(type(c.settings))

The physical system is comprised of components, which are stored in a list ``c.system.cmp_list``

In [ ]:
print(f'cmp_list is a {type(c.system.cmp_list)}')
print(f'cmp_list has length {len(c.system.cmp_list)}')

Let's print information about the components,

In [ ]:
for i in range(3):
    
    print(f'Information about component {i}:')

    # extract component i from the component list
    component = c.system.cmp_list[i]

    # print the name
    print(f'   name =  {component.name}')

    # print a list of the names of the parameters of this component
    parameters = component.parameters
    parameter_names = [par0.name for par0 in parameters]
    string = '   has parameters : '
    for name in parameter_names:
        string += f'{name} , '
    print(string)

    # print the type of this component
    print(f'   type =  {type(component)}')

    # does it contribute to the potential?
    print(f'   contributes to the potential? -  {component.contributes_to_potential}')

Each component has a name, some parameters, and a type. **Currently, dynamite only supports this exact combination of component types**, i.e.

- one ``Plummer``  component representing the black hole
- one component representing the dark halo (DYNAMITE supports ``NFW`` and a number of additional dark halos, see API documentation)
- one ``TriaxialVisibleComponent`` representing the stellar body of the galaxy

In the future, we want to support other components, and more flexible combinations of components.

For the stars - i.e. component 2 - we must provide some input data files. The location of these files is specified in the configuration file, at
```
settings -> io_settings -> input_directory
```
which takes the value,

In [ ]:
c.settings.io_settings['input_directory']

The names of the following files are also specified in the configuration file, in the locations
```
system_components -> stars -> mge_pot
system_components -> stars -> mge_lum
system_components -> stars -> kinematics -> <kinematics name> --> datafile
system_components -> stars -> kinematics -> <kinematics name> --> aperturefile
system_components -> stars -> kinematics -> <kinematics name> --> binfile
```
which take values of the appropriate filenames. You are free to give these files whatever name you like, as long as it is specified in the configuration file.

Let's have a look at the MGE,

In [ ]:
c.system.cmp_list[2].mge_pot, c.system.cmp_list[2].mge_lum

The kinematics are stored here,

In [ ]:
type(c.system.cmp_list[2].kinematic_data)

Note that this object has type ``list``. This is because a single component can have multiple different sets of kinematics. In this example, the first (and only) entry in the list is

In [ ]:
type(c.system.cmp_list[2].kinematic_data[0])

We see that this kinematics object has type ``GaussHermite``. This has also been specified in the configuration file, under
```
system_components -> stars -> kinematics --> kinset1 --> type
```
In addition to ``GaussHermite``, DYNAMITE also supports ``BayesLOSVD`` kinematics (see documentation).

The kinemtic data itself can be accessed as follows,

In [ ]:
c.system.cmp_list[2].kinematic_data[0].data

## Creating a Schwarzschild model

Our next step will be to create a model, i.e. a `dyn.model.Model` object for a particular `parset`. This is a particular set of values for each of the parameters of the model. In the configuration file, every parameter has a been given a `value`. We can extract and inspect a parameter set specified by these values as follows,

In [ ]:
parset = c.parspace.get_parset()
print(parset)

Note that, compared to values specified in the configuration file, parameters which have been specified as logarithmic, i.e. those configured as 
```
parameters -> XXX -> logarithmic : True
```
have been exponentiated in this table. More details can be found in the tutorial/documentation "Parameter Space" (`5_parameter_space.ipynb`).

DYNAMITE uses the `ModelIterator` object to run models. In our case, only one model is run because the configuration file fixes all parameters to their respective values (`fixed: True`). In a later tutorial "Model Iterations and Plots" (`3_model_iterations_and_plots.ipynb`) we will see how to run a suite of models. To help understand what the model iterator does, let's read the internal documentation (i.e. the docstring) for the class:
```
class ModelIterator(object):
    """Iterator for models

    Creating this ``ModelIterator`` object will (i) generate parameters sets,
    (ii) run models for those parameters, (iii) check stopping criteria, and
    iterate this procedure till a stopping criterion is met. This is implemented
    by creating a ``ModelInnerIterator`` object whose ``run_iteration`` method
    is called a number of times.

    Parameters
    ----------
    config : a ``dyn.config_reader.Configuration`` object
    model_kwargs : dict
        other kewyord argument required for this model
    do_dummy_run : Bool
        whether this is a dummy run - if so, dummy_chi2_funciton is executed
        instead of the model (for testing!)
    dummy_chi2_function : function
        a function of model parameters to be executed instead of the real model
    plots : bool
        whether or not to make plots

    """
    def __init__(self,
                 config=None,
                 model_kwargs={},
                 do_dummy_run=None,
                 dummy_chi2_function=None,
                 plots=True):
```
All we need to do is pass the configuration object `c` when we instantiate the `ModelIterator`. The following step will take a few minutes (we won't need the `ModelIterator` object itself, so we discard the return value):

In [ ]:
_ = dyn.model_iterator.ModelIterator(config=c)

A lot happened here: when run for the first time, the particular `parset` is new, so DYNAMITE creates a new entry in AllModels.table and assigns a model directory that will hold the results. The next step is to calculate the orbit library and then DYNAMITE finds out which orbits are useful for reproducing the observations. This is an Non-Negative Least Squares (NNLS) optimization problem resulting in a certain chi-squared value. Let's first have a look at the all models table,

In [ ]:
c.all_models.table

This table is stored in `all_models_file` in the `output_directory` as defined in the configuration file:

In [ ]:
c.settings.io_settings['output_directory'], c.settings.io_settings['all_models_file']

In the output directory we find the all models table and a `models/` subfolder (in our case, `NGC6278_output/all_models.ecsv` and `NGC6278_output/models/`, respectively) which hold the model parameters, status, and results in the so-called model directories as stated in the all models table.

Let's look into how to access information about specific models in the all models table (see the Model API documentation for more information). We start with instantiating a `dyn.model.Model` object from the table (we use the only model in the table which has the index 0):

In [ ]:
model = c.all_models.get_model_from_row(0)

In [ ]:
model.parset # verify the model's parameter set

The model's directory name is constructed from the standard name for new models and their ml value. Here, we are at the first iteration `000` of the ModelIterator and at the first model `000` inside this iteration:

In [ ]:
model.directory

Congratulations! You have run your first Schwarzschild model using DYNAMITE. The chi-squared of this model is,

In [ ]:
model.get_weights() # Read weights if already calculated. If not, calculate weights.
model.chi2

Is that good? I don't know! To find out, we'll have to run more models and compare their chi-squared values. For information about this, see the tutorial "Model Iterations and Plots" (`3_model_iterations_and_plots.ipynb`).

## Plot the Models

Now let's look at some output for the model that we have just run. You might have noticed that the `ModelIterator` also created a directory for plots,

In [ ]:
c.settings.io_settings['plot_directory']

that holds diagnostic plots. In a run with multiple models these plots will be upated as the calculations progress and reflect the latest stete in model parameters, the chi-squared values, and the best-fit kinematic maps.

Here, we'd like to show how to plot kinematic maps for our specific model. After instantiating a `dyn.Plotter` object, we call the appropriate plotting method on our model. Please refer to the Plotting API documentation for more information.

In [ ]:
plotter = dyn.plotter.Plotter(config=c)

figure = plotter.plot_kinematic_maps(model)

The top row shows the data, the middle row shows the model, and the bottom row shows the residuals. The columns, from left to right, are the stellar surface density, $V$, $\sigma$, $h_3$ and $h_4$. We can see the following features in the fit,

- the model and data surface densities are very similar
- the sense of rotation of the $v$ map is reproduced well, though the amplitude is lower than observed
- the $\sigma$, $h_3$ and $h_4$ maps are less well reproduced

While the fit is certainly not perfect, it's reassuring to see that some features are reproduced well. To improve the fit, we will have to explore parameter space more fully. See the tutorial on "running a grid of models" for more details.

## Exercise

Change any parameter you would like, then re-run a model and save the output plots.

Change one of the potential parameters, then run another Schwarzschild model. You could edit the existing or create a new configuration file, delete the contents of the output directory `NGC6278_output`, and re-run the appropriate commands in this notebook.